In [ ]:
from datetime import timedelta
import healpy
import huracanpy
import intake
from iris.analysis.cartography import wrap_lons
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
tracks = huracanpy.load("../hk25-TropCyc/TC_tracks/um_glm_n2560_RAL3p3.csv")
track = tracks.hrcn.sel_id(72)
track

In [ ]:
# Set up initial points. 5-degree circle around cyclone centre
x0 = track.lon.values[0]
y0 = track.lat.values[0]

angle = np.arange(360)

x0 = x0 + 5 * np.sin(np.deg2rad(angle))
y0 = y0 + 5 * np.cos(np.deg2rad(angle))

plt.plot(x0, y0)

In [ ]:
cat = intake.open_catalog('https://digital-earths-global-hackathon.github.io/catalog/catalog.yaml')['online']
ds = cat["um_glm_n2560_RAL3p3"](zoom=9, time="PT3H").to_dask().sel(pressure=200)
ds

In [ ]:
dy = 1.112e5  # Distance in m between 2 lat circles

def update_position(x, y, u, v, dt):
    # Calculate new positions
    x = x + u * dt / (dy * np.cos(np.deg2rad(y)))
    y = y + v * dt / dy

    # Crosses pole
    cross_north_pole = (y > 90)
    y[cross_north_pole] = 180 - y[cross_north_pole]

    cross_south_pole = (y < -90)
    y[cross_south_pole] = -180 + y[cross_south_pole]

    # Check boundaries
    x[cross_north_pole | cross_south_pole] += 180
    x = wrap_lons(x, base=-180, period=360)

    return x, y

# Default iterations from Lagranto
euler_iterations=3
times = pd.date_range(
    start=track.time.values[0],
    end=track.time.values[-1],
    freq="3h",
)
ntimes = len(times)

# Timestep in seconds (for 3-hourly data)
dt = 3 * 3600

# Preallocate array for trajectories and add initial positions
ntraj = len(x0)
trajectories = np.zeros([ntimes + 1, ntraj, 2])
trajectories[0, :, 0] = x0
trajectories[0, :, 1] = y0


for n, time in tqdm(enumerate(times), total=ntimes):
    # First iteration (x0, y0) == (x1, y1)
    x1 = x0
    y1 = y0

    ds_t = ds.sel(time=time)
    ds_tp1 = ds.sel(time=time + timedelta(seconds=dt))

    # Winds at t0
    u0 = healpy.get_interp_val(ds_t.ua, x0, y0, nest=True, lonlat=True)
    v0 = healpy.get_interp_val(ds_t.va, x0, y0, nest=True, lonlat=True)

    # Further iterations. Keep updating x1, y1
    for i in range(euler_iterations - 1):
        # Winds at t+dt
        u1 = healpy.get_interp_val(ds_tp1.ua, x1, y1, nest=True, lonlat=True)
        v1 = healpy.get_interp_val(ds_tp1.va, x1, y1, nest=True, lonlat=True)

        # Average velocity across times
        u = (u0 + u1) / 2.
        v = (v0 + v1) / 2.

        # Calculate new positions
        x1, y1 = update_position(x0, y0, u, v, dt)

    # Add final position to trajectories
    trajectories[n + 1, :, 0] = x1
    trajectories[n + 1, :, 1] = y1

    x0, y0 = x1, y1

np.save("um_n2560_glm_ral3p3_tc72_outflow_trajectories.npy", trajectories)
ds.close()